In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Specify each path

In [2]:
dataset = 'model/keypoint.csv'
model_save_path = 'model/aslr_model.hdf5'
tflite_save_path = 'model/aslr_model.tflite'

# Set number of classes

In [3]:
NUM_CLASSES = 24

# Dataset reading

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [7]:
y_dataset

array([ 0,  0,  0, ..., 16, 16, 16])

# Model building

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [9]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 24)                264       
                                                                 
Total params: 1,334
Trainable params: 1,334
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [11]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [12]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
128/141 [==========================>...] - ETA: 0s - loss: 3.0525 - accuracy: 0.1022
Epoch 1: saving model to model\test-aslr_model.hdf5
141/141 [==============================] - 2s 5ms/step - loss: 3.0319 - accuracy: 0.1074 - val_loss: 2.7401 - val_accuracy: 0.2038
Epoch 2/1000
120/141 [========================>.....] - ETA: 0s - loss: 2.6319 - accuracy: 0.1893
Epoch 2: saving model to model\test-aslr_model.hdf5
141/141 [==============================] - 0s 3ms/step - loss: 2.6016 - accuracy: 0.1969 - val_loss: 2.2563 - val_accuracy: 0.3720
Epoch 3/1000
138/141 [============================>.] - ETA: 0s - loss: 2.2459 - accuracy: 0.2666
Epoch 3: saving model to model\test-aslr_model.hdf5
141/141 [==============================] - 0s 3ms/step - loss: 2.2410 - accuracy: 0.2673 - val_loss: 1.7797 - val_accuracy: 0.5235
Epoch 4/1000
136/141 [===========================>..] - ETA: 0s - loss: 1.9506 - accuracy: 0.3316
Epoch 4: saving model to model\test-aslr_model.hdf5
141/141

In [13]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

47/47 [==============================] - 0s 2ms/step - loss: 0.4310 - accuracy: 0.8632


In [14]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [15]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 103ms/step
[2.0440075e-30 4.6989371e-08 9.9994218e-01 0.0000000e+00 0.0000000e+00
 2.8033337e-19 1.0000000e+00 1.0000000e+00 1.8767435e-14 7.4331364e-36
 7.8054103e-15 0.0000000e+00 0.0000000e+00 7.4803400e-01 2.3781386e-06
 9.9999946e-01 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.8913225e-31
 2.1604797e-19 0.0000000e+00 9.9999946e-01 2.1557693e-09]
6


# Confusion matrix

In [16]:
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)


188/188 [==============================] - 0s 1ms/step


188/188 [==============================] - 0s 1ms/step


Accuracy   : 0.786


In [17]:
from sklearn.metrics import accuracy_score
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y"]
prediction = model.predict(X_test)
y_pred = np.argmax(prediction, axis=1)


cm = confusion_matrix(y_test, y_pred)

#Plot the confusion matrix.
plt.subplots(figsize=(12, 9))

sns.heatmap(cm,
            annot=True,
            fmt='g',
            xticklabels=labels,
            yticklabels=labels)

plt.title('Confusion Matrix',fontsize=17)
# plt.ylabel('Prediction',fontsize=13)
# plt.xlabel('Actual',fontsize=13)
plt.show()


# Finding precision and recall
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)


# Convert to model for Tensorflow-Lite

In [18]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [19]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\Sam\AppData\Local\Temp\tmpkap03ukg\assets


INFO:tensorflow:Assets written to: C:\Users\Sam\AppData\Local\Temp\tmpkap03ukg\assets


7392